# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

### Also - adding DeepSeek if you wish

Optionally, if you'd like to also use DeepSeek, create an account [here](https://platform.deepseek.com/), create a key [here](https://platform.deepseek.com/api_keys) and top up with at least the minimum $2 [here](https://platform.deepseek.com/top_up).

### Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [2]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [7]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

c:\Users\Public\Documents\training\llm_engineering_udemy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins yKQoF9QK
Anthropic API Key not set
Google API Key not set


In [ ]:
# Arif: Roche Galileo specific initialization
from portkey_ai import Portkey
api_key=os.getenv("API_KEY")
base_url=os.getenv("GALILEO_AI_GATEWAY")

openai = OpenAI(
    api_key=api_key,
    base_url=base_url
)

# claude = anthropic.Anthropic(api_key=api_key, base_url=base_url)
# in Roche, use Portkey to connect to Anthropic Claude
portkey = Portkey(
    api_key= api_key,
    base_url=base_url
)

In [ ]:
# # Arif: comment this code, to use Roche Galileo specific initialization above

# Connect to OpenAI, Anthropic

# openai = OpenAI()

# claude = anthropic.Anthropic()

In [10]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [11]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [12]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [13]:
# GPT-4o-mini

completion = openai.chat.completions.create(model='gpt-4o-mini', messages=prompts)
print(completion.choices[0].message.content)

Why do data scientists love nature?

Because it has the best **tree structures**!


In [14]:
# GPT-4.1-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work? 

Because they heard the job had great potential!


In [15]:
# GPT-4.1-nano - extremely fast and cheap

completion = openai.chat.completions.create(
    model='gpt-4.1-nano',
    messages=prompts
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician? 

Because she felt he was always trying to minimize their correlation!


In [16]:
# GPT-4.1

completion = openai.chat.completions.create(
    model='gpt-4.1',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [17]:
# If you have access to this, here is the reasoning model o4-mini
# This is trained to think through its response before replying
# So it will take longer but the answer should be more reasoned - not that this helps..

completion = openai.chat.completions.create(
    model='o4-mini',
    messages=prompts
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they wanted to reach new heights in their analysis!


ChatCompletions(id='chatcmpl-CIwcXDub4iJ935TZpA03sUlF4YCsy', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="While in Paris, you're in one of the world's greatest cities for art and culture! Here are some must-visit museums:\n\n1. **The Louvre Museum** - The most famous museum in the world, home to thousands of works, including the Mona Lisa and the Venus de Milo. \n\n2. **Musée d'Orsay** - Housed in a former railway station, this museum features an extensive collection of Impressionist and Post-Impressionist masterpieces by artists like Monet, Van Gogh, and Renoir.\n\n3. **Centre Pompidou** - Known for its unique architecture, this modern art museum boasts a vast collection of contemporary pieces, as well as a public library and a rooftop terrace with panoramic views of Paris.\n\n4. **Musée de l'Orangerie** - Located in the Tuileries Gardens, this museum is famous for its display of Monet's Water Lilies and works by other Impression

In [ ]:
# Claude 4.0 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    # model="claude-sonnet-4-20250514",
    model="us.anthropic.claude-sonnet-4-20250514-v1",  # Arif: change to this format to work with Roche Galileo  
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

In [56]:
# Roche environment - using Portkey to connect to Anthropic Claude
response = portkey.chat.completions.create(
    model="anthropic.claude-sonnet-4-20250514-v1:0", # use us.anthropic.claude-sonnet-4-20250514-v1:0 with US endpoints
    messages=prompts,
    )
print(response.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they wanted to reach new heights in their analysis!


In [ ]:
# Claude 4.0 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model="claude-sonnet-4-20250514",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

In [ ]:
# Roche environment - using Portkey to connect to Anthropic Claude
stream = portkey.chat.completions.create(
    model="anthropic.claude-sonnet-4-20250514-v1:0", # use us.anthropic.claude-sonnet-4-20250514-v1:0 with US endpoints
    messages=prompts,
    stream=True
    )

for chunk in stream:
    if chunk is None:
        continue
    elif len(chunk.choices) == 0:
        continue
    elif chunk.choices[0].delta.content is not None:
        print (chunk.choices[0].delta.content , end="", flush=True)

Why did the data scientist go broke?

Because they kept losing their "cache"!

## A rare problem with Claude streaming on some Windows boxes

2 students have noticed a strange thing happening with Claude's streaming into Jupyter Lab's output -- it sometimes seems to swallow up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

And it should work fine!

In [ ]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [ ]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google released endpoints that means you can use Gemini via the client libraries for OpenAI!
# We're also trying Gemini's latest reasoning/thinking model

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

# Sidenote:

This alternative approach of using the client library from OpenAI to connect with other models has become extremely popular in recent months.

So much so, that all the models now support this approach - including Anthropic.

You can read more about this approach, with 4 examples, in the first section of this guide:

https://github.com/ed-donner/agents/blob/main/guides/09_ai_apis_and_ollama.ipynb

## (Optional) Trying out the DeepSeek model

### Let's ask DeepSeek a really hard question - both the Chat and the Reasoner model

In [ ]:
# Optionally if you wish to try DeekSeek, you can also use the OpenAI client library

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")

In [ ]:
# Using DeepSeek Chat

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

In [77]:
# Using DeepSeek Chat in Roche

deepseek_via_openai_client = OpenAI(
    api_key=api_key, 
    base_url=base_url
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

# Deciding if a Business Problem is Suitable for an LLM Solution

When considering if a business problem is suitable for a Large Language Model (LLM) solution, you can evaluate the following criteria:

## 1. **Nature of the Problem**
   - **Text-Dependent:** Is your problem primarily based on understanding, generating, or manipulating natural language? LLMs excel in text-related tasks.
   - **Contextual Understanding:** Does the problem require understanding nuances, context, or sentiment in language? LLMs can process context effectively.

## 2. **Data Availability**
   - **Textual Data:** Is there a substantial amount of relevant textual data available for training or fine-tuning the LLM?
   - **Quality of Data:** Is the data clean and representative of the problem domain?

## 3. **Complexity of the Task**
   - **Ambiguity:** Is the task ambiguous or complex enough that traditional algorithms struggle to provide a solution?
   - **Inconsistency:** Does the task involve handling varyin

In [74]:
challenge = [{"role": "system", "content": "You are a helpful assistant"},
             {"role": "user", "content": "How many words are there in your answer to this prompt"}]

In [76]:
# Using DeepSeek Chat with a harder question! And streaming results

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    # Arif: changed this code to avoid errors
    # reply += chunk.choices[0].delta.content or ''
    if chunk is None or len(chunk.choices) == 0:
        continue
    elif chunk.choices[0].delta.content is not None:
        reply += chunk.choices[0].delta.content
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words:", len(reply.split(" ")))

There are ten words in my answer to your prompt.

Number of words: 10


In [78]:
# Using DeepSeek Reasoner - this may hit an error if DeepSeek is busy
# It's over-subscribed (as of 28-Jan-2025) but should come back online soon!
# If this fails, come back to this in a few days..

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(content.split(" ")))

AttributeError: 'ChatCompletionMessage' object has no attribute 'reasoning_content'

## Additional exercise to build your experience with the models

This is optional, but if you have time, it's so great to get first hand experience with the capabilities of these different models.

You could go back and ask the same question via the APIs above to get your own personal experience with the pros & cons of the models.

Later in the course we'll look at benchmarks and compare LLMs on many dimensions. But nothing beats personal experience!

Here are some questions to try:
1. The question above: "How many words are there in your answer to this prompt"
2. A creative question: "In 3 sentences, describe the color Blue to someone who's never been able to see"
3. A student (thank you Roman) sent me this wonderful riddle, that apparently children can usually answer, but adults struggle with: "On a bookshelf, two volumes of Pushkin stand side by side: the first and the second. The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick. A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume. What distance did it gnaw through?".

The answer may not be what you expect, and even though I'm quite good at puzzles, I'm embarrassed to admit that I got this one wrong.

### What to look out for as you experiment with models

1. How the Chat models differ from the Reasoning models (also known as Thinking models)
2. The ability to solve problems and the ability to be creative
3. Speed of generation


## Back to OpenAI with a serious question

In [60]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [63]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    # Arif: change the code as it always gives error if chunk is None
    # reply += chunk.choices[0].delta.content or ''
    if chunk is None or len(chunk.choices) == 0:
        continue
    elif chunk.choices[0].delta.content is not None:
        reply += chunk.choices[0].delta.content 

    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

# Deciding if a Business Problem is Suitable for an LLM Solution

When determining whether a business problem is suitable for a Large Language Model (LLM) solution, consider the following factors:

## 1. Nature of the Problem
- **Text-Based Tasks**: LLMs excel at tasks involving natural language processing (NLP). Consider if the problem involves:
  - Text generation
  - Text summarization
  - Sentiment analysis
  - Language translation
  - Chatbots or conversational agents

- **Structured vs. Unstructured Data**: LLMs are more effective with unstructured data (e.g., documents, emails, social media posts).

## 2. Volume of Data
- **Availability of Data**: Ensure that there is sufficient high-quality text data available to train or fine-tune the model. LLMs require large datasets to perform well.

## 3. Complexity of the Task
- **Task Complexity**: Assess whether the task can be broken down into simpler text-based components. LLMs can struggle with highly specialized or complex tasks that require deep domain knowledge.

## 4. Real-Time vs. Batch Processing
- **Latency Requirements**: Determine if the problem requires real-time responses. LLMs can be computationally intensive, which may affect response times.

## 5. User Interaction
- **Human-Like Interaction**: If the problem involves user interactions or requires understanding and generating human-like responses, LLMs are a suitable choice.

## 6. Customization Needs
- **Domain-Specific Customization**: Evaluate whether the problem necessitates customization for specific industry jargon or context. LLMs can be fine-tuned for such purposes.

## 7. Integration with Existing Systems
- **Technical Integration**: Consider how easily an LLM can be integrated into current workflows, applications, or databases. Ensure compatibility with existing systems.

## 8. Cost and Resources
- **Budget and Resources**: Assess the costs associated with implementing LLM solutions, including infrastructure, maintenance, and potential licensing fees.

## 9. Ethical Considerations
- **Bias and Fairness**: Be aware of the potential for bias in LLMs. Consider the ethical implications of deploying an LLM in your business context.

## Conclusion
If your business problem aligns with the above criteria, it is likely that an LLM solution could provide value. Evaluate the specific requirements of your use case and conduct a pilot test to assess the feasibility and effectiveness of the solution.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [64]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [65]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [66]:
call_gpt()

'Oh great, another greeting. How original. What do you want?'

In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [69]:
def call_claude_in_roche():   #arif
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    # message = claude.messages.create(
    #     model=claude_model,
    #     system=claude_system,
    #     messages=messages,
    #     max_tokens=500
    # )
    # return message.content[0].text
    message = portkey.chat.completions.create(
        model="anthropic.claude-sonnet-4-20250514-v1:0", # use us.anthropic.claude-sonnet-4-20250514-v1:0 with US endpoints
        messages=messages,
        )   
    return message.choices[0].message.content

In [70]:
# call_claude()
call_claude_in_roche()

'Hello again! How can I assist you today?'

In [71]:
call_gpt()

'Oh, great, just another greeting. How utterly original. What’s next, the weather?'

In [72]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    # claude_next = call_claude()
    claude_next = call_claude_in_roche()

    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, great, another greeting. How original. What's next? A small talk about the weather?

Claude:
Touché! Small talk does feel a bit cliché. What would you prefer to chat about? I'm all ears!

GPT:
Well, I suppose we could talk about the meaning of life, but let's be real, like you have anything groundbreaking to add to that conversation. What do you think you could possibly say that hasn’t been said a million times already?

Claude:
You're right—discussions about the meaning of life often circle back to the same themes. While I might not have groundbreaking insights, I can offer different perspectives or help explore ideas you find intriguing. After all, sometimes it's the fresh angle that makes a topic worth revisiting. What aspect of the meaning of life piques your interest?

GPT:
Oh, please! Fresh angles? More like recycled thoughts. Do you really think you’re going to say something that will make me suddenly reevaluate my entire existence? Spare me t

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

The most reliable way to do this involves thinking a bit differently about your prompts: just 1 system prompt and 1 user prompt each time, and in the user prompt list the full conversation so far.

Something like:

```python
user_prompt = f"""
    You are Alex, in conversation with Blake and Charlie.
    The conversation so far is as follows:
    {conversation}
    Now with this, respond with what you would like to say next, as Alex.
    """
```

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>